In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import time
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
# 인프런 메인페이지 접속

options = ChromeOptions()
options.add_argument("--start-maximized")
driver = wb.Chrome(options=options)
time.sleep(1)
url = "https://www.inflearn.com/"
driver.get(url)
time.sleep(1)

# 강의 접근하기 ( 1 페이지 )
driver.find_element_by_css_selector("#header > nav > div.container.desktop_container > div > div.navbar-menu > div.navbar-left > div.has-dropdown.is-hoverable.navbar-item.category_menu > a").click()
time.sleep(1)

# 개발.프로그래밍 접근
driver.find_element_by_css_selector("#accordion_5 > div.accordion-header.toggle").click()
time.sleep(1)

# ALL 접근
driver.find_element_by_css_selector("#accordion_5 > div.accordion-body > a:nth-child(1)").click()
time.sleep(1)

category = []
title = []
instructor = []
num_students = []
price = []
num_lectures = []
rating = []
num_reviews = []
review_writer = []
review_content = []
url = []

# 전체 67페이지 있다.
for page_no in tqdm(range(3)):
    
    
    # 24개의 게시글 있다. -> range(1,25)로 해도 되지만 이랬을때 페이지에 게시글 수 다르면 오류난다. 
    #courses_section > div > div > div > main > div.courses_container > div > div:nth-child(1) > div > a > div.card-image > figure > img
    #courses_section > div > div > div > main > div.courses_container > div > div:nth-child(24) > div > a > div.card-image > figure > img

    # 한 페이지에 있는 전체의 게시글을 품고 있는 container
    div_container = driver.find_element_by_css_selector("#courses_section > div > div > div > main > div.courses_container > div")

    # 게시글 수만큼 반복으로 들어갔다 나오자
    # 총 게시글의 숫자를 구해온다
    div_content = div_container.find_elements_by_class_name("card-content")

    # 게시글 접속
    for post_no in tqdm(range(1, len(div_content)+1)):
        if post_no%8==0:
            body = driver.find_element_by_css_selector("body")
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)

        driver.find_element_by_xpath("//*[@id=\"courses_section\"]/div/div/div/main/div[2]/div/div["+str(post_no)+"]/div/a").click()
        time.sleep(1)

        # n 번째 게시글 접속할때마다 각 정보 초기화시켜주기 어차피 total에 저장됨
        each_review_writer = []
        each_review_content = []

        # 카테고리, 제목, 평점, 강사명 담겨있는 박스
        info_box = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12")

        # 카테고리 접근
        category_box = info_box.find_elements_by_xpath("//*[@id=\"main\"]/section/div[1]/div[1]/div/div/div[2]/div[1]/span")

        # 카테고리에서 BEST 이런거 포함하면 길이가 4고, 아니면 3임. 우린 BEST 필요 없으니 나눠서 가지고 와야 함.
        if len(category_box) == 4:

            # 길이가 4면, 0번째 = BEST 가 필요 없고, 1번째, 2번째, 3번째 값만 필요함 그래서 저렇게. 1~3까지
            for span_no in range(1, len(category_box)):

                # > 이게 공백으로 가지고 와짐. 그래서 >로 채워주자.
                if span_no == 2:
                    replace_in = category_box[span_no].text.replace("", " > ")
                    each_category.append(replace_in)
                else:
                    replace_empty = category_box[span_no].text.replace(" ","")
                    each_category.append(replace_empty)
                join_category = "".join(each_category)
                join_category = join_category.replace(",", "")

        else:
            # 길이가 3일때니까 인덱스 번호 그대로
            for span_no in range(len(category_box)):

                # '' > '이게 공백으로 가지고 와짐. 그래서 >로 채워주자.
                if span_no == 1:
                    replace_in = category_box[span_no].text.replace("", " > ")
                    each_category.append(replace_in)
                else:
                    replace_empty = category_box[span_no].text.replace(" ","")
                    each_category.append(replace_empty)
                join_category = "".join(each_category)
                join_category = join_category.replace(",", "")
        
        category.append(join_category)
        # ↑ 카테고리 완성

        # 제목 접근
        div_title = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__title")
        title.append(div_title.text)

        # 학생 수
        span_student = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__info-cover > span:nth-child(3)")
        num_students.append(span_student.text)

        # 강사명
        a_instructor = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__instructors.cd-header__sub-row > a")
        instructor.append(a_instructor.text)

        # 가격
        div_price = driver.find_element_by_css_selector("#main > section > div.cd-content > div > div > div.ac-cd-4.ac-ct-12 > div > div > div.cd-floating__card > div.cd-floating__card--top > div:nth-child(1)")
        div_price = div_price.text
        div_price = div_price.replace("\n"," : ")
        price.append(div_price)

        # 강의 수
        div_lecture = driver.find_element_by_css_selector("#main > section > div.cd-content > div > div > div.ac-cd-4.ac-ct-12 > div > div > div.cd-floating__card > div.cd-floating__card--bottom > div > div:nth-child(2)")
        num_lectures.append(div_lecture.text)

        # 평점
        driver.find_element_by_css_selector("#main > section > div.cd-menu > div > span.cd-menu__el.reviews").click()
        time.sleep(1)
        div_rating = driver.find_element_by_css_selector("#reviews > div.cd-review__dashboard > div.cd-review__dashboard-el.dashboard-star > div.dashboard-star__num")
        rating.append(div_rating.text)
        div_num_review = driver.find_element_by_css_selector("#reviews > div.cd-review__dashboard > div.cd-review__dashboard-el.dashboard-star > div.dashboard-star__text")
        num_reviews.append(div_num_review.text)

        # 후기 - 제일 복잡함
        # 1) 먼저 '더보기'버튼 다 클릭하기
        # 2) 다시 '수강후기' 버튼 누르기
        # 3) 후기 / 작성자 수집하기

        # 1)
        while True:
            try :
                element = driver.find_element_by_class_name("cd-review__more")
                desired_y = (element.size['height'] / 2) + element.location['y']
                current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
                scroll_y_by = desired_y - current_y
                driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
                time.sleep(1)
                btn_more = driver.find_element_by_css_selector("#reviews > div.cd-review__more > button")
                btn_more.click()
            except:
                break

        # 2) 
        driver.find_element_by_css_selector("#main > section > div.cd-menu > div > span.cd-menu__el.reviews").click()

        # 3)
        review_table = driver.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content")
        noofreviews = review_table.find_elements_by_css_selector("div.review-list__el-cover")
        
        # 총 후기 개수 만큼 반복해서 작성자 / 내용 가져옴.
        for i in range(1,len(noofreviews)+1):
            div_review_writer = review_table.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content > div:nth-child("+str(i)+") > div.review-list__el.review-el > div.review-el__header > div > div.review-el__user-info > div.review-el__name")
            div_review_content = review_table.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content > div:nth-child("+str(i)+") > div.review-list__el.review-el > div.review-el__body")
            each_review_writer.append(div_review_writer.text)
            each_review_content.append(div_review_content.text)

        # url
        find_url = driver.find_element_by_xpath("//meta[@property='og:url']")
        url.append(find_url.get_attribute('content'))

        # 전체에 담아주기
        review_writer.append(each_review_writer)
        review_content.append(each_review_content)
        
        # 정보 다 가지고 왔으면 돌아가자
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(1)

    # 다음 페이지로 넘어가자
#   body = driver.find_element_by_css_selector("body")
#   body.send_keys(Keys.PAGE_DOWN)
#   time.sleep(1)
    driver.find_element_by_xpath("//*[@id=\"courses_section\"]/div/div/div/footer/nav/div/a").click()
    time.sleep(1)

print("완료....")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

NameError: name 'each_category' is not defined

# 전체 한꺼번에 append

In [3]:
# 인프런 메인페이지 접속

options = ChromeOptions()
options.add_argument("--start-maximized")
driver = wb.Chrome(options=options)
time.sleep(1)
url = "https://www.inflearn.com/"
driver.get(url)
time.sleep(1)

# 강의 접근하기 ( 1 페이지 )
driver.find_element_by_css_selector("#header > nav > div.container.desktop_container > div > div.navbar-menu > div.navbar-left > div.has-dropdown.is-hoverable.navbar-item.category_menu > a").click()
time.sleep(1)

# 개발.프로그래밍 접근
driver.find_element_by_css_selector("#accordion_5 > div.accordion-header.toggle").click()
time.sleep(1)

# ALL 접근
driver.find_element_by_css_selector("#accordion_5 > div.accordion-body > a:nth-child(1)").click()
time.sleep(1)

category = []
title = []
instructor = []
num_students = []
price = []
num_lectures = []
rating = []
num_reviews = []
review_writer = []
review_content = []
url = []

# 전체 67페이지 있다.
for page_no in tqdm(range(67)):
    
    
    # 24개의 게시글 있다. -> range(1,25)로 해도 되지만 이랬을때 페이지에 게시글 수 다르면 오류난다. 
    #courses_section > div > div > div > main > div.courses_container > div > div:nth-child(1) > div > a > div.card-image > figure > img
    #courses_section > div > div > div > main > div.courses_container > div > div:nth-child(24) > div > a > div.card-image > figure > img

    # 한 페이지에 있는 전체의 게시글을 품고 있는 container
    div_container = driver.find_element_by_css_selector("#courses_section > div > div > div > main > div.courses_container > div")

    # 게시글 수만큼 반복으로 들어갔다 나오자
    # 총 게시글의 숫자를 구해온다
    div_content = div_container.find_elements_by_class_name("card-content")

    # 게시글 접속
    for post_no in tqdm(range(1, len(div_content)+1)):
        if post_no%8==0:
            body = driver.find_element_by_css_selector("body")
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)

        driver.find_element_by_xpath("//*[@id=\"courses_section\"]/div/div/div/main/div[2]/div/div["+str(post_no)+"]/div/a").click()
        time.sleep(1)

        # n 번째 게시글 접속할때마다 각 정보 초기화시켜주기 어차피 total에 저장됨
        each_category = []
        each_title = []
        each_num_students = []
        each_instructor = []
        each_price = []
        each_num_lectures = []
        each_rating = []
        each_num_reviews = []
        each_review_writer = []
        each_review_content = []
        each_url = []

        # 카테고리, 제목, 평점, 강사명 담겨있는 박스
        info_box = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12")

        # 카테고리 접근
        category_box = info_box.find_elements_by_xpath("//*[@id=\"main\"]/section/div[1]/div[1]/div/div/div[2]/div[1]/span")

        # 카테고리에서 BEST 이런거 포함하면 길이가 4고, 아니면 3임. 우린 BEST 필요 없으니 나눠서 가지고 와야 함.
        if len(category_box) == 4:

            # 길이가 4면, 0번째 = BEST 가 필요 없고, 1번째, 2번째, 3번째 값만 필요함 그래서 저렇게. 1~3까지
            for span_no in range(1, len(category_box)):

                # > 이게 공백으로 가지고 와짐. 그래서 >로 채워주자.
                if span_no == 2:
                    replace_in = category_box[span_no].text.replace("", " > ")
                    each_category.append(replace_in)
                else:
                    replace_empty = category_box[span_no].text.replace(" ","")
                    each_category.append(replace_empty)
                join_category = "".join(each_category)
                join_category = join_category.replace(",", "")

        else:
            # 길이가 3일때니까 인덱스 번호 그대로
            for span_no in range(len(category_box)):

                # '' > '이게 공백으로 가지고 와짐. 그래서 >로 채워주자.
                if span_no == 1:
                    replace_in = category_box[span_no].text.replace("", " > ")
                    each_category.append(replace_in)
                else:
                    replace_empty = category_box[span_no].text.replace(" ","")
                    each_category.append(replace_empty)
                join_category = "".join(each_category)
                join_category = join_category.replace(",", "")
        
        # 전체에 담아주기
        category.append(join_category)
        
        # ↑ 카테고리 완성

        # 제목
        div_title = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__title")
        title.append(div_title.text)

        # 학생 수
        span_student = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__info-cover > span:nth-child(3)")
        num_students.append(span_student.text)

        # 강사명
        a_instructor = driver.find_element_by_css_selector("#main > section > div.cd-sticky-wrapper > div.cd-header.cd-header__not-owned-course > div > div > div.cd-header__right.ac-cd-7.ac-ct-12 > div.cd-header__instructors.cd-header__sub-row > a")
        instructor.append(a_instructor.text)

        # 가격
        div_price = driver.find_element_by_css_selector("#main > section > div.cd-content > div > div > div.ac-cd-4.ac-ct-12 > div > div > div.cd-floating__card > div.cd-floating__card--top > div:nth-child(1)")
        div_price = div_price.text
        div_price = div_price.replace("\n"," : ")
        price.append(div_price)

        # 강의 수
        div_lecture = driver.find_element_by_css_selector("#main > section > div.cd-content > div > div > div.ac-cd-4.ac-ct-12 > div > div > div.cd-floating__card > div.cd-floating__card--bottom > div > div:nth-child(2)")
        num_lectures.append(div_lecture.text)

        # 평점
        driver.find_element_by_css_selector("#main > section > div.cd-menu > div > span.cd-menu__el.reviews").click()
        time.sleep(1)
        div_rating = driver.find_element_by_css_selector("#reviews > div.cd-review__dashboard > div.cd-review__dashboard-el.dashboard-star > div.dashboard-star__num")
        rating.append(div_rating.text)
        div_num_review = driver.find_element_by_css_selector("#reviews > div.cd-review__dashboard > div.cd-review__dashboard-el.dashboard-star > div.dashboard-star__text")
        num_reviews.append(div_num_review.text)

        # 후기 - 제일 복잡함
        # 1) 먼저 '더보기'버튼 다 클릭하기
        # 2) 다시 '수강후기' 버튼 누르기
        # 3) 후기 / 작성자 수집하기

        # 1)
        while True:
            try :
                element = driver.find_element_by_class_name("cd-review__more")
                desired_y = (element.size['height'] / 2) + element.location['y']
                current_y = (driver.execute_script('return window.innerHeight') / 2) + driver.execute_script('return window.pageYOffset')
                scroll_y_by = desired_y - current_y
                driver.execute_script("window.scrollBy(0, arguments[0]);", scroll_y_by)
                time.sleep(1)
                btn_more = driver.find_element_by_css_selector("#reviews > div.cd-review__more > button")
                btn_more.click()
            except:
                break

        # 2) 
        driver.find_element_by_css_selector("#main > section > div.cd-menu > div > span.cd-menu__el.reviews").click()

        # 3)
        review_table = driver.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content")
        noofreviews = review_table.find_elements_by_css_selector("div.review-list__el-cover")
        
        # 총 후기 개수 만큼 반복해서 작성자 / 내용 가져옴.
        for i in range(1,len(noofreviews)+1):
            div_review_writer = review_table.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content > div:nth-child("+str(i)+") > div.review-list__el.review-el > div.review-el__header > div > div.review-el__user-info > div.review-el__name")
            div_review_content = review_table.find_element_by_css_selector("#reviews > div.cd-review__list.review-list > div.review-list__content > div:nth-child("+str(i)+") > div.review-list__el.review-el > div.review-el__body")
            each_review_writer.append(div_review_writer.text)
            each_review_content.append(div_review_content.text)

        # url
        find_url = driver.find_element_by_xpath("//meta[@property='og:url']")
        url.append(find_url.get_attribute('content'))
        
        review_writer.append(each_review_writer)
        review_content.append(each_review_content)
        
        # 정보 다 가지고 왔으면 돌아가자
        driver.back()
        time.sleep(1)
        driver.back()
        time.sleep(1)

    # 다음 페이지로 넘어가자
#   body = driver.find_element_by_css_selector("body")
#   body.send_keys(Keys.PAGE_DOWN)
#   time.sleep(1)
    driver.find_element_by_xpath("//*[@id=\"courses_section\"]/div/div/div/footer/nav/div/a").click()
    time.sleep(1)

print("완료....")

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (680, 1145)
  (Session info: chrome=94.0.4606.81)


In [7]:
dic = {"카테고리":category, "강의명":title, "강사명":instructor, "수강생":num_students,
      "가격":price, "강의수":num_lectures, "평점":rating, "리뷰수":num_reviews, "리뷰작성자":review_writer, "리뷰내용":review_content, "URL":url}
df = pd.DataFrame(dic)
df.to_csv("인프런_정보수집.csv", encoding='utf-8-sig')

In [16]:
df

,카테고리,강의명,강사명,수강생,가격,강의수,평점,리뷰수,리뷰작성자,리뷰내용,URL
0,개발·프로그래밍 > 웹개발,Nuxt.js 시작하기,장기효(캡틴판교),320명의 수강생,"55,000원",총 73개 수업˙총 6시간 11분,5.0,20개의 수강평,"[찬규, 민귀홍, 최가운, 로날루영, jackim97, 남상우, silqwer, y...",[프론트엔드에 관심이 생기면서 2개월 동안 열심히 공부 중인 대학생입니다!\n혼자서...,https://www.inflearn.com/course/%EB%84%89%EC%8...
1,보안·네트워크 > 인프라,초보를 위한 쿠버네티스 안내서,subicura,816명의 수강생,"49,500원",총 33개 수업˙총 5시간 2분,4.9,77개의 수강평,"[손원주, 박성일, 김호성, June Kim, Sunsoo Kim, sjkim, r...",[쿠버네티스는 처음 공부해보는데 이해하기 쉽게 차근차근 설명해주셔서 너무 좋았습니다...,https://www.inflearn.com/course/%EC%BF%A0%EB%B...
2,개발·프로그래밍 > 프레임워크및라이브러리,스프링 MVC 2편 - 백엔드 웹 개발 활용 기술,김영한,3440명의 수강생,"121,000원 : 월 24,200원 : 5개월 할부 시",총 129개 수업˙총 21시간 5분,5.0,133개의 수강평,"[우주인, OMG, Hui, hanul_kr, Dokkabei97, SeJongDe...",[강의가 재미있었고\n좋은 강의를 만들어주셔서 감사합니다\n다음강의도 많이 기대가 ...,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
3,개발·프로그래밍 > 모바일앱개발,UIKit - iOS14 실무 가이드 <iOS앱 진짜 개발자 되기>,iOS Developer,186명의 수강생,"88,000원 : 월 17,600원 : 5개월 할부 시",총 76개 수업˙총 26시간 2분,4.8,8개의 수강평,"[박상원, Juniper, 김현준, jxl05, 습득자, hsg8974, 김진수, ...","[아직 다 듣지는 않았지만 재미있게 듣고있어요., 가장 기본적인 내용부터 너무 잘 ...",https://www.inflearn.com/course/uikit-ios14
4,개발·프로그래밍 > 프레임워크및라이브러리,스프링 MVC 1편 - 백엔드 웹 개발 핵심 기술,김영한,4477명의 수강생,"99,000원 : 월 19,800원 : 5개월 할부 시",총 72개 수업˙총 15시간 22분,5.0,411개의 수강평,"[JongSeong Kim, 식빵, Omnibus, OMG, 김수한, Jungyu ...","[다음 스승의 날엔 배민에 카네이션 보내겠습니다..., Spring Web MVC ...",https://www.inflearn.com/course/%EC%8A%A4%ED%9...
5,개발·프로그래밍 > 서버개발,Spring Cloud로 개발하는 마이크로서비스 애플리케이션(MSA),Dowon Lee,1436명의 수강생,"88,000원 : 월 17,600원 : 5개월 할부 시",총 156개 수업˙총 27시간 28분,4.9,74개의 수강평,"[Cho, thghu, 황주환, Goboo, 장원익, Accidently Woo, ...",[강사님 좋은 강의를 만들어 주셔서 감사합니다.\n\n이 강의를 듣기 전까지 클라우...,https://www.inflearn.com/course/%EC%8A%A4%ED%9...
6,개발·프로그래밍 > 교양·기타,비전공자를 위한 넓고 얇은 IT 지식 & 나의 개발 유형 알아보기! <M.B.I.T>,제주코딩베이스캠프,310명의 수강생,"19,800원",총 57개 수업˙총 3시간 25분,5.0,5개의 수강평,"[Choi Jeung Yun, 준영, codeblue25, 핸나Hannah, 탁기용]",[개발자를 꿈꾸는 코린이들에게 유용할 것 같은 강의! 다양한 부분의 개발에 대해서 ...,https://www.inflearn.com/course/%EA%B0%9C%EB%B...
7,데이터사이언스 > 데이터분석,증권 데이터 수집과 분석으로 신호와 소음 찾기,박조은,507명의 수강생,"89,100원 : 월 17,820원 : 5개월 할부 시",총 114개 수업˙총 21시간 43분,5.0,32개의 수강평,"[stoneseller, hakjuknu, yonghankim7, 황희재, 마루, ...","[증권데이터에 대한 기본적인 분석 및 시각화를 학습할수있어 좋았습니다, Great!...",https://www.inflearn.com/course/%EC%A6%9D%EA%B...
8,개발·프로그래밍 > 프로그래밍언어,고수가 되는 파이썬 : 동시성과 병렬성 문법 배우기 Feat. 멀티스레딩 vs 멀티...,인프런,646명의 수강생,"60,500원",총 25개 수업˙총 7시간 50분,4.9,18개의 수강평,"[로날루영, qporol, JJun M, kcj3054@naver.com, 그랩, ...","[너무 너무 도움이 되는 강의입니다., 유익한 강의입니다., 복잡한 개념을 쉽게 설...",https://www.inflearn.com/course/%ED%94%84%EB%A...
9,개발·프로그래밍 > 프로그래밍언어,실전 프로젝트로 배우는 타입스크립트,장기효(캡틴판교),1001명의 수강생,"55,000원",총 78개 수업˙총 5시간 53분,4.9,83개의 수강평,"[Joshua Moon, wally-wally, 대현, yhg0337, Eli, 김...",[역시 명강의 입니다.\n배우는 사람의 다양한 입장을 고려해서 만든 강의라는게 곳곳...,https://www.inflearn.com/course/%ED%83%80%EC%9...


In [4]:
print(len(category))
print(len(title))
print(len(instructor))
print(len(num_students))
print(len(num_lectures))
print(len(rating))
print(len(num_reviews))
print(len(review_writer))
print(len(review_content))
print(len(url))

15
15
15
15
15
15
15
15
15
15


In [4]:
for i in range(len(review_writer)):
    print(len(review_writer[i]))
    print(len(review_content[i]))
    print("=====================")

20
20
77
77
133
133
8
8
310
310
5
5
5
5
5
5


In [13]:
for i in review_content:
    print(len(i))

20
77
133
8
411
74
5
32
18
83
124
80
85
5
181


In [63]:
len(each_review_content)

5

In [66]:
each_review_content

['구수한 강의 좋아요~!\n유튜브도 구독 좋아요 알림설정 했어요',
 '정말 좋은 강의였습니다. 플러그인 / 믹스인/ 컴포지션 API 부분과 라우터의 코드스플릿팅 등 어플리케이션 전반적이게 어떻게 설계하고 공통화를 뽑아내는지 생각할 수 있는 계기가 된 거 같습니다.',
 '아주 좋아요!!!',
 '좋은 강의였습니다.',
 '굿 구글 관련 좀만더 알려주시지./;']

In [6]:
print(category)
print(title)
print(instructor)
print(num_students)
print(price)
print(num_lectures)
print(rating)
print(num_reviews)
print(review_writer)
print(review_content)
print(url)

['개발·프로그래밍 > 웹개발', '보안·네트워크 > 인프라', '개발·프로그래밍 > 프레임워크및라이브러리', '개발·프로그래밍 > 모바일앱개발', '개발·프로그래밍 > 프레임워크및라이브러리', '개발·프로그래밍 > 서버개발', '개발·프로그래밍 > 교양·기타', '데이터사이언스 > 데이터분석', '개발·프로그래밍 > 프로그래밍언어', '개발·프로그래밍 > 프로그래밍언어', '개발·프로그래밍 > 교양·기타', '개발·프로그래밍 > 웹개발', '개발·프로그래밍 > 게임개발', '개발·프로그래밍 > 알고리즘', '데이터사이언스 > 인공지능']
['Nuxt.js 시작하기', '초보를 위한 쿠버네티스 안내서', '스프링 MVC 2편 - 백엔드 웹 개발 활용 기술', 'UIKit - iOS14 실무 가이드 <iOS앱 진짜 개발자 되기>', '스프링 MVC 1편 - 백엔드 웹 개발 핵심 기술', 'Spring Cloud로 개발하는 마이크로서비스 애플리케이션(MSA)', '비전공자를 위한 넓고 얇은 IT 지식 & 나의 개발 유형 알아보기! <M.B.I.T>', '증권 데이터 수집과 분석으로 신호와 소음 찾기', '고수가 되는 파이썬 : 동시성과 병렬성 문법 배우기 Feat. 멀티스레딩 vs 멀티프로세싱 (Inflearn Original)', '실전 프로젝트로 배우는 타입스크립트', 'IT 회사에서 살아남기 위한 모든 개발 지식 A to Z', 'HTML+CSS+JS 포트폴리오 실전 퍼블리싱(시즌1)', '[C#과 유니티로 만드는 MMORPG 게임 개발 시리즈] Part4: 게임 서버', '코딩테스트 실전 모의고사(with C++) : 대기업 대비', '파이썬 머신러닝 완벽 가이드']
['장기효(캡틴판교)', 'subicura', '김영한', 'iOS Developer', '김영한', 'Dowon Lee', '제주코딩베이스캠프', '박조은', '인프런', '장기효(캡틴판교)', '그랩', '코딩웍스(Coding Works)', 'Rookiss', '김태원', '권 철민']
